## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-13-21-51-10 +0000,bbc,Two women shot dead at Kentucky church,https://www.bbc.com/news/articles/cm2l5jn6254o
1,2025-07-13-21-40-33 +0000,startribune,Three Twin Cities suburbs under boil water adv...,https://www.startribune.com/boil-water-advisor...
2,2025-07-13-21-39-17 +0000,startribune,Twins head into All-Star break under .500 with...,https://www.startribune.com/minnesota-twins-he...
3,2025-07-13-21-37-33 +0000,bbc,Nigeria and Algeria progress at Wafcon after draw,https://www.bbc.com/sport/football/articles/cl...
4,2025-07-13-21-21-06 +0000,bbc,Everything you need to know about Wafcon 2024 ...,https://www.bbc.com/sport/football/articles/cq...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
133,against,4
44,trump,4
10,under,3
108,risks,3
24,wafcon,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
35,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...,24
34,2025-07-13-01-04-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...,20
15,2025-07-13-16-57-56 +0000,nyt,How Trump’s Latest Tariff Threat Risks Deeper ...,https://www.nytimes.com/2025/07/13/business/tr...,16
19,2025-07-13-16-29-05 +0000,bbc,Gaza officials say children killed in strike a...,https://www.bbc.com/news/articles/c0rvxjnvv71o,16
9,2025-07-13-20-28-06 +0000,nyt,Children Among at Least 10 Killed in Israeli S...,https://www.nytimes.com/2025/07/13/world/middl...,15


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
35,24,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...
19,16,2025-07-13-16-29-05 +0000,bbc,Gaza officials say children killed in strike a...,https://www.bbc.com/news/articles/c0rvxjnvv71o
15,16,2025-07-13-16-57-56 +0000,nyt,How Trump’s Latest Tariff Threat Risks Deeper ...,https://www.nytimes.com/2025/07/13/business/tr...
26,15,2025-07-13-13-00-00 +0000,startribune,Tonka Bay begins moving mounds of accidentally...,https://www.startribune.com/tonka-bay-sacred-s...
28,14,2025-07-13-11-00-00 +0000,startribune,A wrongful-death ruling at an assisted-living ...,https://www.startribune.com/a-wrongful-death-r...
21,13,2025-07-13-16-13-14 +0000,nyt,"In Canada’s Northern Outposts, Rusting Relics ...",https://www.nytimes.com/2025/07/13/world/ameri...
25,12,2025-07-13-13-00-00 +0000,startribune,"Brown: At 5,000 feet, the legacy of Iron Range...",https://www.startribune.com/iron-range-mining-...
30,12,2025-07-13-08-00-00 +0000,cbc,Half of requests for complex dental work are b...,https://www.cbc.ca/news/politics/dental-care-e...
34,12,2025-07-13-01-04-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...
38,11,2025-07-12-23-40-23 +0000,bbc,Inside the Congolese mine vital to mobile phon...,https://www.bbc.com/news/articles/cyvj986l615o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
